In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

2024-09-23 20:52:43.636561: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-23 20:52:43.655987: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-23 20:52:43.847729: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-23 20:52:44.096639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 20:52:44.245780: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [3]:
df = pd.read_csv("train.csv")

In [5]:
#Retirando coluna id do usuario
df.drop(['id'], axis=1, inplace=True)
df_train.drop(['id'], axis=1, inplace=True)

label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])
Y_train = df.iloc[:, 0].values.astype(np.float32)

df.drop(['class'], axis=1, inplace=True)


In [7]:
print(df.nunique(axis=0))

cap-diameter            3913
cap-shape                 74
cap-surface               83
cap-color                 78
does-bruise-or-bleed      26
gill-attachment           78
gill-spacing              48
gill-color                63
stem-height             2749
stem-width              5836
stem-root                 38
stem-surface              60
stem-color                59
veil-type                 22
veil-color                24
has-ring                  23
ring-type                 40
spore-print-color         32
habitat                   52
season                     4
dtype: int64


In [8]:
#removendo dados duplicados
dublicados = df.duplicated().sum()
print(dublicados)

0


### defido a quantidade de dados faltantes seria inadequado a remocao dos devidos dados. Entao uma viavel solucao seria a criacao de uma nova classe chamada desconhecido.

In [9]:
#verificando se possui coluna vazia
df.isnull().sum().sort_values(ascending=False).head(20)

veil-type               2957493
spore-print-color       2849682
stem-root               2757023
veil-color              2740947
stem-surface            1980861
gill-spacing            1258435
cap-surface              671023
gill-attachment          523936
ring-type                128880
gill-color                   57
habitat                      45
cap-shape                    40
stem-color                   38
has-ring                     24
cap-color                    12
does-bruise-or-bleed          8
cap-diameter                  4
stem-width                    0
stem-height                   0
season                        0
dtype: int64

In [10]:
coln_null = df.columns[df_train.isnull().any() & (df.dtypes != 'float64') & (df.dtypes != 'int64')]
coln_null_train = df_train.columns[df_train.isnull().any() & (df_train.dtypes != 'float64') & (df_train.dtypes != 'int64')]
print(coln_null_train)

Index(['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
       'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root',
       'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring',
       'ring-type', 'spore-print-color', 'habitat'],
      dtype='object')


In [11]:
df[coln_null] = df[coln_null].fillna('not_registered')
df_train[coln_null_train] = df_train[coln_null_train].fillna('not_registered')

In [15]:
df.dtypes

cap-diameter      float64
stem-height       float64
stem-width        float64
cap-shape_0.82       bool
cap-shape_1.66       bool
                   ...   
habitat_z            bool
season_a             bool
season_s             bool
season_u             bool
season_w             bool
Length: 823, dtype: object

In [12]:
#fazendo one hot em todas as colunas que tiver mais de duas variaveis
'''
    Seleciona as colunas de strings do DataFrame usando select_dtypes
    pd.get_dummies(): Aplica o One-Hot Encoding diretamente nessas colunas.
'''

df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns)


In [ ]:
# Seleciona todas as colunas de características, exceto a coluna de saída (coluna 8)
X_train = df.iloc[:, :822].values.astype(np.float32)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.25)

In [ ]:
classificador = Sequential()
classificador.add(Dense(units =1645, activation = 'relu', input_dim = 822))
classificador.add(Dense(units = 822, activation = 'relu'))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dense(units = 1, activation = 'sigmoid'))
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['binary_accuracy'])
classificador.summary()

In [ ]:
history = classificador.fit(X_train, Y_train, batch_size = 200,epochs = 20, verbose = 1, validation_split=0.2)

In [ ]:
resultado = classificador.evaluate(X_test, Y_test, verbose=1)

previsoes = classificador.predict(X_test)

previsoes = (previsoes > 0.5).astype(int)

classe_teste = (Y_test > 0.5).astype(int)

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(classe_teste, previsoes)

print(matriz)
